In [29]:
import pandas as pd

class Vivienda:
    def __init__(self, deuda_inicial, interes_anual, valor_inicial, plazo, anio_compra, cashflow=0, alquiler=0):
        self.deuda_inicial = deuda_inicial
        self.interes_anual = interes_anual / 100 if interes_anual > 1 else interes_anual
        self.valor_inicial = valor_inicial
        self.plazo = plazo
        self.anio_compra = anio_compra
        self.cashflow_inicial = cashflow
        self.alquiler_inicial = alquiler

    def deuda_restante(self, anio_actual):
        anios_transcurridos = max(0, min(anio_actual - self.anio_compra, self.plazo))
        r = self.interes_anual
        n = self.plazo
        if anios_transcurridos == 0:
            return self.deuda_inicial
        deuda_rest = self.deuda_inicial * ((1 + r)**n - (1 + r)**anios_transcurridos) / ((1 + r)**n - 1)
        return deuda_rest

    def valores_actuales(self, anio_actual, subida_alquiler):
        anios_transcurridos = max(0, anio_actual - self.anio_compra)
        factor = (1 + subida_alquiler) ** anios_transcurridos
        alquiler = self.alquiler_inicial * factor
        cashflow = self.cashflow_inicial * factor
        return alquiler, cashflow

    def cuota_anual(self, anio_actual, subida_alquiler):
        alquiler, cashflow = self.valores_actuales(anio_actual, subida_alquiler)
        return (alquiler - cashflow) * 12

    def cashflow_anual(self, anio_actual, subida_alquiler):
        _, cashflow = self.valores_actuales(anio_actual, subida_alquiler)
        return cashflow * 12

    def valor_futuro(self, anio_actual, revalorizacion_anual):
        anios_transcurridos = max(0, anio_actual - self.anio_compra)
        return self.valor_inicial * ((1 + revalorizacion_anual) ** anios_transcurridos)

In [25]:
ahorro_anual = 8000
ahorro_inicial = 60000

vivienda1 = Vivienda(deuda_inicial=72000, interes_anual=0.019, valor_inicial=80000, plazo=40, cashflow=418, alquiler=650, anio_compra=2025)
vivienda2 = Vivienda(deuda_inicial=64000, interes_anual=0.019, valor_inicial=80000, plazo=35, cashflow=420, alquiler=650, anio_compra=2026)
vivienda3 = Vivienda(deuda_inicial=48000, interes_anual=0.019, valor_inicial=80000, plazo=35, cashflow=447, alquiler=650, anio_compra=2027)
vivienda4 = Vivienda(deuda_inicial=48000, interes_anual=0.019, valor_inicial=80000, plazo=35, cashflow=447, alquiler=650, anio_compra=2029)
vivienda5 = Vivienda(deuda_inicial=48000, interes_anual=0.019, valor_inicial=80000, plazo=35, cashflow=447, alquiler=650, anio_compra=2030)
vivienda6 = Vivienda(deuda_inicial=48000, interes_anual=0.019, valor_inicial=80000, plazo=35, cashflow=447, alquiler=650, anio_compra=2032)
vivienda7 = Vivienda(deuda_inicial=48000, interes_anual=0.019, valor_inicial=80000, plazo=35, cashflow=447, alquiler=650, anio_compra=2033)
vivienda8 = Vivienda(deuda_inicial=48000, interes_anual=0.019, valor_inicial=80000, plazo=35, cashflow=447, alquiler=650, anio_compra=2034)
vivienda9 = Vivienda(deuda_inicial=48000, interes_anual=0.019, valor_inicial=80000, plazo=35, cashflow=447, alquiler=650, anio_compra=2035)

viviendas = [vivienda1, vivienda2, vivienda3, vivienda4, vivienda5, vivienda6, vivienda7, vivienda8, vivienda9]

In [24]:
def simular_patrimonio(anio_inicio, anio_final, ahorro_inicial, ahorro_anual, tasa_interes, revalorizacion,
                       tipo1, tipo2, tipo3, sueldo_mensual):
    
    viviendas = []
    ahorro = ahorro_inicial
    resultados = []
    sueldo_anual = sueldo_mensual * 12

    for anio in range(anio_inicio, anio_final + 1):
        ahorro += ahorro_anual

        # Ingresos por alquiler (cashflow neto)
        cashflow_total = sum(v.cashflow_anual() for v in viviendas if v.anio_compra <= anio)
        ahorro += cashflow_total

        # Bucle de compra (permite más de una compra por año a partir del 5º año)
        while True:
            n = len(viviendas)
            if n == 0:
                tipo = tipo1
            elif n == 1:
                tipo = tipo2
            else:
                tipo = tipo3

            valor = tipo["valor"]
            ratio_fin = tipo["ratio_financiacion"]
            entrada = tipo["entrada"]

            if ahorro < entrada:
                break
            if anio - anio_inicio < 5 and any(v.anio_compra == anio for v in viviendas):
                break

            deuda = valor * ratio_fin
            plazo = tipo["plazo"]
            ahorro -= entrada
            viviendas.append(Vivienda(deuda, tasa_interes, valor, plazo, anio, tipo["cashflow"], tipo["alquiler"]))

            if anio - anio_inicio < 5:
                break

        # Calcular valores agregados
        valor_total = sum(v.valor_futuro(anio, revalorizacion) for v in viviendas)
        deuda_total = sum(v.deuda_restante(anio) for v in viviendas)
        # Pagos hipotecarios totales (usando la simplificación requerida)
        pagos_hipoteca_total = sum(v.cuota_anual() for v in viviendas if v.anio_compra <= anio)

        # Ingresos totales = alquiler (cashflow) + sueldo
        ingresos_totales = cashflow_total + sueldo_anual

        # Ratio de endeudamiento real
        ratio_endeudamiento = pagos_hipoteca_total / ingresos_totales if ingresos_totales > 0 else None

        net_value = ahorro + (valor_total - deuda_total)

        resultados.append({
            "Año": anio,
            "Ahorro (€)": ahorro,
            "Valor total inmuebles (€)": valor_total,
            "Deuda total (€)": deuda_total,
            "Patrimonio neto (€)": net_value,
            "Cashflow total (€)": cashflow_total,
            "Pagos hipoteca (anuales simpl.) (€)": pagos_hipoteca_total,
            "Ingresos totales (€)": ingresos_totales,
            "Ratio endeudamiento": ratio_endeudamiento,
            "Viviendas activas": len(viviendas)
        })

    return pd.DataFrame(resultados)

In [27]:
df = evolucion_net_value(viviendas, ahorro_inicial=60000, anio_inicio=2025, anio_final=2035, ahorro_anual=8000)

df

,Año,Ahorro restante (€),Valor total inmuebles (€),Deuda total (€),Patrimonio neto (€),Cashflow total anual (€)
0,2025,51016,"80,000.00","72,000.00","59,016.00",5016
1,2026,39072,"165,600.00","134,781.93","69,890.07",10056
2,2027,24492,"257,192.00","180,236.54","101,447.46",15420
3,2028,47912,"275,195.44","176,664.68","146,442.76",15420
4,2029,38696,"374,459.12","221,024.94","192,130.18",20784
5,2030,34844,"480,671.26","264,337.93","251,177.33",26148
6,2031,68992,"514,318.25","258,583.74","324,726.51",26148
7,2032,70504,"630,320.52","300,720.22","400,104.30",31512
8,2033,77380,"754,442.96","341,767.17","490,055.79",36876
9,2034,89620,"887,253.97","381,703.89","595,170.08",42240


In [30]:
def simular_patrimonio(anio_inicio, anio_final, ahorro_inicial, ahorro_anual, tasa_interes, revalorizacion,
                       tipo1, tipo2, tipo3, sueldo_mensual, subida_alquiler=0.015):
    
    viviendas = []
    ahorro = ahorro_inicial
    resultados = []
    sueldo_anual = sueldo_mensual * 12

    for anio in range(anio_inicio, anio_final + 1):
        ahorro += ahorro_anual

        # Ingresos por alquiler actualizados
        cashflow_total = sum(v.cashflow_anual(anio, subida_alquiler) for v in viviendas if v.anio_compra <= anio)
        ahorro += cashflow_total

        # Bucle de compra (más de una compra a partir del 5º año)
        while True:
            n = len(viviendas)
            if n == 0:
                tipo = tipo1
            elif n == 1:
                tipo = tipo2
            else:
                tipo = tipo3

            valor = tipo["valor"]
            ratio_fin = tipo["ratio_financiacion"]
            entrada = tipo["entrada"]

            if ahorro < entrada:
                break
            if anio - anio_inicio < 5 and any(v.anio_compra == anio for v in viviendas):
                break

            deuda = valor * ratio_fin
            plazo = tipo["plazo"]
            ahorro -= entrada
            viviendas.append(Vivienda(deuda, tasa_interes, valor, plazo, anio, tipo["cashflow"], tipo["alquiler"]))

            if anio - anio_inicio < 5:
                break

        # Cálculos anuales
        valor_total = sum(v.valor_futuro(anio, revalorizacion) for v in viviendas)
        deuda_total = sum(v.deuda_restante(anio) for v in viviendas)
        pagos_hipoteca_total = sum(v.cuota_anual(anio, subida_alquiler) for v in viviendas if v.anio_compra <= anio)

        ingresos_totales = cashflow_total + sueldo_anual
        ratio_endeudamiento = pagos_hipoteca_total / ingresos_totales if ingresos_totales > 0 else None

        net_value = ahorro + (valor_total - deuda_total)

        resultados.append({
            "Año": anio,
            "Ahorro (€)": ahorro,
            "Valor total inmuebles (€)": valor_total,
            "Deuda total (€)": deuda_total,
            "Patrimonio neto (€)": net_value,
            "Cashflow total (€)": cashflow_total,
            "Pagos hipoteca (anuales simpl.) (€)": pagos_hipoteca_total,
            "Ingresos totales (€)": ingresos_totales,
            "Ratio endeudamiento": ratio_endeudamiento,
            "Viviendas activas": len(viviendas)
        })

    return pd.DataFrame(resultados)

In [31]:
tipo1 = {"entrada": 22000, "valor": 80000, "plazo": 40, "cashflow": 400, "alquiler": 650, "ratio_financiacion": 0.90}
tipo2 = {"entrada": 30000, "valor": 80000, "plazo": 35, "cashflow": 420, "alquiler": 650, "ratio_financiacion": 0.80}
tipo3 = {"entrada": 38000, "valor": 80000, "plazo": 30, "cashflow": 447, "alquiler": 650, "ratio_financiacion": 0.70}

df = simular_patrimonio(
    anio_inicio=2025,
    anio_final=2045,
    ahorro_inicial=60000,
    ahorro_anual=8000,
    tasa_interes=1.9,
    revalorizacion=0.02,
    tipo1=tipo1,
    tipo2=tipo2,
    tipo3=tipo3,
    sueldo_mensual=2000,
    subida_alquiler=0.015
)

pd.set_option('display.float_format', '{:,.2f}'.format)
df

,Año,Ahorro (€),Valor total inmuebles (€),Deuda total (€),Patrimonio neto (€),Cashflow total (€),Pagos hipoteca (anuales simpl.) (€),Ingresos totales (€),Ratio endeudamiento,Viviendas activas
0,2025,"46,000.00","80,000.00","72,000.00","54,000.00",0.00,"3,000.00","24,000.00",0.12,1
1,2026,"28,872.00","161,600.00","134,781.93","55,690.07","4,872.00","5,805.00","28,872.00",0.20,2
2,2027,"8,932.68","244,832.00","188,236.54","65,528.14","10,060.68","8,328.07","34,060.68",0.24,3
3,2028,"32,588.73","249,728.64","184,240.65","98,076.72","15,656.05","8,453.00","39,656.05",0.21,3
4,2029,"18,479.62","334,723.21","236,168.84","117,033.99","15,890.89","11,015.79","39,890.89",0.28,4
5,2030,"10,053.34","421,417.68","286,617.52","144,853.49","21,573.71","13,617.03","45,573.71",0.30,5
6,2031,"7,395.12","509,846.03","335,558.57","181,682.57","27,341.78","16,257.28","51,341.78",0.32,6
7,2032,"10,591.48","600,042.95","382,963.36","227,671.07","33,196.37","18,937.14","57,196.37",0.33,7
8,2033,"19,730.25","692,043.81","428,802.70","282,971.37","39,138.77","21,657.20","63,138.77",0.34,8
9,2034,"34,900.57","785,884.69","473,046.83","347,738.42","45,170.31","24,418.06","69,170.31",0.35,9
